In [1]:
import numpy as np
import pandas as pd

In [2]:
import torch
import sklearn
import random
import torchvision
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# setting seed
torch.manual_seed(1)
if device == "cuda":
    torch.cuda.manual_seed_all(1)
random.seed(1)

In [4]:
# load data 
train = pd.read_csv("./2021-ai-w10-p1/train.csv")
test = pd.read_csv("./2021-ai-w10-p1/test.csv")
submission = pd.read_csv("./2021-ai-w10-p1/sample_submit.csv")

print(train.info())
print(submission.head())
print(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 785 entries, Category to 28x28
dtypes: int64(785)
memory usage: 359.3 MB
None
   Id  Category
0   0         0
1   1         0
2   2         0
3   3         0
4   4         0
   Category  1x1  1x2  1x3  1x4  1x5  1x6  1x7  1x8  1x9  ...  28x19  28x20  \
0         5    0    0    0    0    0    0    0    0    0  ...      0      0   
1         0    0    0    0    0    0    0    0    0    0  ...      0      0   
2         3    0    0    0    0    0    0    0    0    0  ...      0      0   
3         3    0    0    0    0    0    0    0    0    0  ...      0      0   
4         6    0    0    0    0    0    0    0    0    0  ...      0      0   

   28x21  28x22  28x23  28x24  28x25  28x26  28x27  28x28  
0      0      0      0      0      0      0      0      0  
1      0      0      0      0      0      0      0      0  
2      0      0      0      0      0      0      0      0  
3      0      0      0    

In [5]:
# data preprocessing

# data scaling
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

y_train = np.array(train['Category'])
x_train = np.array(train.drop(['Category'],axis=1))
x_test = np.array(test)

x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [6]:
# set data on Tensor
x_train = torch.Tensor(x_train).to(device)
y_train = torch.LongTensor(y_train).to(device)
x_test = torch.Tensor(x_test).to(device)

print(x_train.shape, x_test.shape)
print(y_train.shape)

torch.Size([60000, 784]) torch.Size([10000, 784])
torch.Size([60000])


In [7]:
# chage data shape 1d -> 2d
x_train = x_train.view(60000,1,28,28)
x_test = x_test.view(10000,1,28,28)
print(x_train.shape)
print(x_test.shape)

torch.Size([60000, 1, 28, 28])
torch.Size([10000, 1, 28, 28])


In [8]:
# set Tensor on Dataset
train_dataset = torch.utils.data.TensorDataset(x_train,y_train)

In [9]:
# pretrain하면 layer가 바뀌지 않는다.
model = torchvision.models.resnet18()
model.conv1 = torch.nn.Conv2d(1,64,kernel_size = (7,7),stride =(2,2), padding=(3, 3), bias=False)
model.fc = torch.nn.Linear(512,10,bias=True)
model = model.to(device)
print(model)

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [10]:
# setting param, optim, cost function,dataloader
from torch.utils.data import DataLoader

lr = 1e-3
batch_size = 10
optim = torch.optim.Adam(model.parameters(), lr=lr)
data_loader = DataLoader(dataset=train_dataset, batch_size = batch_size, shuffle=True)
epochs = 5
loss = torch.nn.CrossEntropyLoss()

In [11]:
##### 메모리가 넘치게 됨 !! 왜???? 함수형 train 일때는 왜 정상작동 일까?





# # learning
# model.train()
# for epoch in range(epochs+1):
#     sum_cost = 0
#     for x,y in data_loader:
#         h = model(x)
#         cost = loss(h,y)
        
#         optim.zero_grad()
#         cost.backward()
#         optim.step()
#         sum_cost += cost
#     if epoch % (epochs/10) == 0 :
#         model.eval()
#         predict = model(x_train)
#         predict = torch.argmax(predict,dim=1)
#         acc = predict == y_train
#         acc = acc.float().mean().cpu().detach().item()
#         print(epoch,sum_cost.item()/len(data_loader),acc)

In [12]:
def train(model, data_loader):
    model.train()
    sum_cost = 0.0
    sum_correct = 0
    for i,data in enumerate(data_loader):
        data, target = data[0],data[1]
        optim.zero_grad()
        output = model(data)
        cost = loss(output,target)
        sum_cost += cost.item()
        predict = torch.argmax(output,dim=1)
        correct = (predict == target).sum().item()
        sum_correct += correct
        cost.backward()
        optim.step()
    return sum_cost/len(data_loader),sum_correct/len(data_loader)

In [13]:
import time
cur_time = time.time()
for epoch in range(epochs):
    cost,acc = train(model,data_loader)
    print(epoch,cost,acc*10)
print("endtime =",time.time() - cur_time)

0 0.20667266537225806 94.18333333333334


KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    model.eval()
    predict = model(x_train)
    predict = torch.argmax(predict,dim=1)
    print(predict)
    acc = y_train == predict
print(acc.float().mean())

In [ ]:
with torch.no_grad():
    model.eval()
    predict = model(x_test)
    predict = torch.argmax(predict,dim=1)
    submission['Category'] = predict.cpu().detach()
    print(submission)

In [ ]:
submission.to_csv("submission.csv",index=False)